<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [110]:
class Node():
    def __init__(self, state, parent, action):
        self.state = state
        self.parent = parent
        self.action = action

In [111]:
class StackFrontier():
    def __init__(self):
        self.frontier = []
    def add(self, node):
        self.frontier.append(node)
    def contains_state(self, state):
        return any(node.state == state for node in self.frontier)
    def empty(self):
        return len(self.frontier) == 0
    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[-1]
            self.frontier = self.frontier[:-1]
            return node

In [112]:
class QueueFrontier(StackFrontier):
    def remove(self):
        if self.empty():
            raise Exception("empty frontier")
        else:
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            return node

In [113]:
class Maze():
    def __init__ (self, filename):
        with open(filename) as f:
            contents = f.read()
        if contents.count("A") != 1:
            raise Exception("maze must have only 1 start point")
        if contents.count("B") != 1:
            raise Exception("maze must have only 1 goal")
        contents = contents.splitlines()
        self.height = len(contents)
        self.width = max(len(line) for line in contents)
        self.walls = []
        for i in range(self.height):
            row = []
            for j in range(self.width):
                try:
                    if contents[i][j] == "A":
                        self.start = (i, j)
                        row.append(False)
                    elif contents[i][j] == "B":
                        self.goal = (i, j)
                        row.append(False)
                    elif contents[i][j] == " ":
                        row.append(False)
                    else:
                        row.append(True)
                except IndexError:
                    row.append(False)
            self.walls.append(row)
        self.solution = None

    def print(self):
        solution = self.solution[1] if self.solution is not None else None
        print()
        for i, row in enumerate(self.walls):
            for j, col in enumerate(row):
                if col:
                    print("█", end="")
                elif (i, j) == self.start:
                    print("A", end="")
                elif(i, j) == self.goal:
                    print("B", end="")
                elif solution is not None and (i, j) in solution:
                    print("*", end="")
                else:
                    print(" ", end="")
            print()
        print()
    
    def neighbors(self, state):
        row, col = state
        candidates = [("up", (row-1, col)),
                     ("down", (row+1, col)),
                     ("left", (row, col-1)),
                     ("right", (row, col+1))]
        result = []
        for action, (r, c) in candidates:
            if 0 <= r < self.height and 0 <= c < self.width and not self.walls[r][c]:
                result.append((action, (r, c)))
        return result
    
    def solve_stack(self):
        self.num_explored = 0
        start = Node(state=self.start, parent=None, action=None)
        frontier = StackFrontier()
        frontier.add(start)
        
        self.explored = set()
        while True:
            if frontier.empty():
                raise Exception("no solution")

            node = frontier.remove()

            self.num_explored += 1

            if node.state ==self.goal:
                actions = []
                cells = []
                while node.parent is not None:
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                actions.reverse()
                cells.reverse()
                self.solution = (actions, cells)
                return

            self.explored.add(node.state)

            for action, state in self.neighbors(node.state):
                if not frontier.contains_state(state) and state not in self.explored:
                    child = Node(state=state, parent=node, action=action)
                    frontier.add(child)

    def solve_queue(self):
        self.num_explored = 0
        start = Node(state=self.start, parent=None, action=None)
        frontier = QueueFrontier()
        frontier.add(start)
        
        self.explored = set()
        while True:
            if frontier.empty():
                raise Exception("no solution")

            node = frontier.remove()

            self.num_explored += 1

            if node.state ==self.goal:
                actions = []
                cells = []
                while node.parent is not None:
                    actions.append(node.action)
                    cells.append(node.state)
                    node = node.parent
                actions.reverse()
                cells.reverse()
                self.solution = (actions, cells)
                return

            self.explored.add(node.state)

            for action, state in self.neighbors(node.state):
                if not frontier.contains_state(state) and state not in self.explored:
                    child = Node(state=state, parent=node, action=action)
                    frontier.add(child)

In [115]:
fname = 'maze4.txt'
m = Maze(fname)
#print("Maze:")
#m.print()
m.solve_stack()
print("Solving by Stack(depth-first search)")
print("States Explored:", m.num_explored)
print("Solution:")
m.print()
print()
m2 = Maze(fname)
m2.solve_queue()
print("Solving by Queue(breadth-first search)")
print("States Explored:", m2.num_explored)
print("Solution:")
m2.print()

Solving by Stack(depth-first search)
States Explored: 160
Solution:

███                 █████████
█   ███████████████████   █ █
█ ████                █ █ █ █
█ ███████████████████ █ █ █ █
█*********************█ █ █ █
█*███████████████████*█ █ █ █
█*  ██****************█ █ █ █
█*█ ██*███ ██ █████████ █ █ █
█*█****█   ██ █         █ █ █
█B█*██ ████████████████ █ █ █
███*██             ████ █ █ █
███*██████████████ ██ █ █ █ █
███****         ██    █ █ █ █
██████*████████ ███████ █ █ █
██████*████             █   █
A******██████████████████████


Solving by Queue(breadth-first search)
States Explored: 52
Solution:

███                 █████████
█   ███████████████████   █ █
█ ████                █ █ █ █
█ ███████████████████ █ █ █ █
█                     █ █ █ █
█ ███████████████████ █ █ █ █
█***██                █ █ █ █
█*█*██ ███ ██ █████████ █ █ █
█*█*   █   ██ █         █ █ █
█B█*██ ████████████████ █ █ █
███*██             ████ █ █ █
███*██████████████ ██ █ █ █ █
███****         ██ 

In [ ]:
print(m.solution)
print(m.num_explored)
print(m.explored)

In [ ]:
# print(m.height)
# print(m.width)
# print(*m.walls, sep="\n")
# print(m.solution)